In [79]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
import sys

In [74]:
class VolData:
    def __init__(self, ticker, option_type='call', n=3):
        self.ticker = ticker
        self.option_type = option_type
        self.n = n

        """
        Initializing Class.
        Parameters
        ----------
        ticker: string 
            The stock ticker of interest        
        option_type: string
            The type of option that will be used to visualize the volatility smile ("call" or "put")
        n: integer
            "n" represents number of options above and below at-the-money (ATM) used to calculate the 
             volatility term structure.For example, assuming $1 strike differences, if n=3 and ATM=100, 
             then the following strikes will be used for calculating the implied volatility term structure:
                 - Lower ATM: 97, 98, 99
                 - ATM: 100
                 - Upper ATM: 101, 102, 103
        
        Example
        ---------
        >>> import yahoo_vol as yvol
        >>> data = yvol.Options(ticker="AAPL", n=3)
        """
        

    def get_input_data(self, save_csv=False):
        """
        Intermediate function used to collect the pricing data for plotting purposes. 
        Parameter:
        ------------
        save_csv: boolean
            Select True to save the options data to a csv file
            
        Returns
        -------------
        dataframe:
            A dataframe with corresponding statistics for each option
        csv:
            A csv file will be saved if save_csv is set to True
        Example
        -------------
        >>> df = data.get_input_data(save_csv=True)
        """
        
        storage = []
        x = yf.Ticker(self.ticker)
        option_dates = x.options
        if len(option_dates) == 0:
            sys.exit("Ticker does not have any options. Please try another ticker")
        for date in tqdm(option_dates):
            try:
                call, put = x.option_chain(date)[0], x.option_chain(date)[1]
                call['option_type'], put['option_type'] = ('call', 'put')
                d = pd.concat([call, put])
                storage.append(d)
            except:
                print(date, "option failed to download")
                pass
        
        print("All option dates successfully downloaded: ", len(storage) == len(option_dates))
        df = pd.concat(storage)
        df = df.reset_index()    
        df['maturity'] = pd.DataFrame([i[-15:-9] for i in df.contractSymbol])
        return df

    def plot_vol_smile(self):
        """
        A plotting function used to visualize the volatility smile for each option maturity by plotting
        the raw implied volatility against the respective strike values. 
            
        Returns
        -------------
        plt.plot figure:
            A series of plots for each option maturity
        Example
        -------------
        >>> data.plot_vol_smile()
        """
        data = self.get_input_data()
        # Extract unique option maturity dates
        maturities = list(set(data.maturity))
        maturities.sort()
        # Loop through each maturity and plot the vol smile 
        for date in maturities:
            temp = data[(data.maturity == date) & (data.option_type == self.option_type)]
            title_date = datetime.date(2000 + int(date[:2]), int(date[2:4]), int(date[4:])).strftime("%B %d, %Y")
            plt.scatter(temp.strike, temp.impliedVolatility)
            plt.title(self.ticker.upper() + " " + self.option_type.upper() + ": " + title_date)
            plt.ylabel("Implied Volatility")
            plt.xlabel("Strike")
            plt.show()


    def plot_vol_term_structure(self):
        data = self.get_input_data()
        p = yf.Ticker(self.ticker).info
        mid_price = (p['ask'] + p['bid'])/2
        strikes = list(set(data.strike))
        maturities = list(set(data.maturity))

        get_closest_strike = min(range(len(strikes)), key=lambda i: abs(strikes[i]- mid_price))
        closest_ATM_indexes = list(range(get_closest_strike - self.n,
                                         get_closest_strike + self.n))
        closest_ATM_strikes = [strikes[i] for i in closest_ATM_indexes]
        storage = []
        for date in maturities:
            try: # NOTE - THE CALLS AND PUTS HERE ARE SEPERATED OUT - LOOK INTO HOW TO INCLUDE BOTH
                temp = data[(data.maturity == date) & (data.option_type== self.option_type)]
                storage.append(temp[temp['strike'].isin(closest_ATM_strikes)].mean()['impliedVolatility'])
            except:
                continue
        df = pd.DataFrame([maturities, storage]).T
        df = df.sort_values(0)
        df = df.dropna()
        plt.plot(df[0], df[1])
        plt.show()

In [75]:
x = VolData("AAPL", option_type='put')

In [99]:
d = requests.get('https://query2.finance.yahoo.com/v7/finance/options/amd').json()

In [106]:
dates = d['optionChain']['result'][0]['expirationDates']

In [112]:
pd.to_datetime(dates, unit='s')

DatetimeIndex(['2020-05-29', '2020-06-05', '2020-06-12', '2020-06-19',
               '2020-06-26', '2020-07-02', '2020-07-17', '2020-10-16',
               '2021-01-15', '2021-03-19', '2021-06-18', '2022-01-21'],
              dtype='datetime64[ns]', freq=None)

In [113]:
dates

[1590710400,
 1591315200,
 1591920000,
 1592524800,
 1593129600,
 1593648000,
 1594944000,
 1602806400,
 1610668800,
 1616112000,
 1623974400,
 1642723200]

In [114]:
d = requests.get('https://query2.finance.yahoo.com/v7/finance/options/amd?date=1591920000').json()

In [98]:
d['optionChain']['result'][0]['options']

[{'expirationDate': 1590710400,
  'hasMiniOptions': False,
  'calls': [{'contractSymbol': 'AMD200529C00025000',
    'strike': 25.0,
    'currency': 'USD',
    'lastPrice': 27.42,
    'change': 0.0,
    'percentChange': 0.0,
    'volume': 4,
    'openInterest': 4,
    'bid': 27.95,
    'ask': 28.2,
    'contractSize': 'REGULAR',
    'expiration': 1590710400,
    'lastTradeDate': 1590586532,
    'impliedVolatility': 1.0000000000000003e-05,
    'inTheMoney': True},
   {'contractSymbol': 'AMD200529C00030000',
    'strike': 30.0,
    'currency': 'USD',
    'lastPrice': 26.95,
    'change': 0.0,
    'percentChange': 0.0,
    'volume': 3,
    'openInterest': 1,
    'bid': 23.1,
    'ask': 23.25,
    'contractSize': 'REGULAR',
    'expiration': 1590710400,
    'lastTradeDate': 1589410803,
    'impliedVolatility': 1.0000000000000003e-05,
    'inTheMoney': True},
   {'contractSymbol': 'AMD200529C00035000',
    'strike': 35.0,
    'currency': 'USD',
    'lastPrice': 18.37,
    'change': 1.3600006

In [ ]:
# need to manually call the dates and manually call the option price data.
# only way to really get full assurance that the package will work. 